# 20 Seed registry
NeuralForecast等のAPI情報の最小サンプルを `registry_*` に投入。

In [1]:
from fcast.meta.db import session_scope, init_db
from fcast.meta.orm import (RegistryLibrary, RegistryRelease, RegistrySymbol, RegistrySignature,
                            RegistryParam, RegistryParamEnum, RegistryParamConstraint,
                            RegistryCapability, RegistrySymbolCapability)
init_db()


In [2]:
from datetime import datetime
with session_scope() as s:
    lib = RegistryLibrary(name="neuralforecast", vendor="Nixtla",
                          repo_url="https://github.com/Nixtla/neuralforecast",
                          docs_url="https://nixtlaverse.nixtla.io/neuralforecast/core.html")
    s.add(lib); s.flush()
    rel = RegistryRelease(library_id=lib.library_id, version="demo", commit_sha=None)
    s.add(rel); s.flush()

    sym = RegistrySymbol(release_id=rel.release_id, kind="method",
                         qualname="neuralforecast.core.NeuralForecast.fit",
                         doc_url="https://nixtlaverse.nixtla.io/neuralforecast/core.html")
    s.add(sym); s.flush()

    sig = RegistrySignature(symbol_id=sym.symbol_id,
                            signature_text="fit(df=None, static_df=None, val_size=0, use_init_models=False, distributed_config=None, prediction_intervals=None, ...)",
                            return_type_repr="self",
                            source="manual")
    s.add(sig); s.flush()

    p = RegistryParam(signature_id=sig.signature_id, name="local_scaler_type", position=0,
                      kind="keyword", required=False, type_repr="str|None", default_repr="None",
                      doc_text="Local scaler type.")
    s.add(p); s.flush()

    # enums from docs
    for v in ["standard","robust","robust-iqr","minmax","boxcox"]:
        s.add(RegistryParamEnum(param_id=p.param_id, value_repr=v, label=v))

    # example constraints
    s.add(RegistryParamConstraint(param_id=p.param_id, constraint_type="enumref",
                                  constraint_json={"values":["standard","robust","robust-iqr","minmax","boxcox"]},
                                  error_message="invalid local_scaler_type"))

    # capabilities
    cap = RegistryCapability(code="exog_futr", description="Accepts future exogenous variables")
    s.add(cap); s.flush()
    s.add(RegistrySymbolCapability(symbol_id=sym.symbol_id, capability_id=cap.capability_id, value_json={"enabled": True}, source="manual"))
print("seeded")


seeded


In [3]:
import pandas as pd
from fcast.meta.db import session_scope
from fcast.meta.orm import RegistryLibrary, RegistryRelease, RegistrySymbol, RegistryParamEnum
with session_scope() as s:
    libs = s.query(RegistryLibrary).all()
    syms = s.query(RegistrySymbol).all()
    enums = s.query(RegistryParamEnum).all()

pd.DataFrame([{"name":x.name,"vendor":x.vendor} for x in libs]), pd.DataFrame([{"qualname":x.qualname} for x in syms]).head(), pd.DataFrame([{"enum":x.value_repr} for x in enums]).head()


DetachedInstanceError: Instance <RegistryLibrary at 0x230d1d01a90> is not bound to a Session; attribute refresh operation cannot proceed (Background on this error at: https://sqlalche.me/e/20/bhk3)